## PDF Query using LangChain

In [2]:
from IPython.display import display as disp

### Loading PDF on langchain document loader 

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
loader = PyPDFLoader("./tmp/tmp.pdf")
pages = loader.load_and_split()
print(f'no of page  = {len(pages)}')
print(f'page_content length = {len(pages[0].page_content)}')

no of page  = 380
page_content length = 192


#### Data Chunks in smaller documents 

In [5]:
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
texts = text_spliter.split_documents(pages)
texts[8]

Document(page_content='be able to reproduce, animals must solve a continuous stream of problems during theirlives, e.g., ﬁnding food, avoiding predators, mating, and parenting. This suggests that\nhuman intelligence primarily evolved for solving everyday problems related to survival\nin the different habitats of Homo sapiens .\nArtiﬁcial Intelligence started as an attempt to reproduce parts of human intelligence\nin machines and, just like the notion of human intelligence, it is associated with a\ncertain vagueness regarding its deﬁnition, targeted problems, performance measures,\nand relations to neighboring research ﬁelds.\nRecently, AI research has been quite successful at producing systems that are gen-\neral in the sense that they can translate between many languages, play many games,\nmanipulate many objects, predict many video frames, write many texts, generate many\nimages, and diagnose many diseases.\nStill, many of the basic challenges of AGI remain unsolved. In fact, we do n

##### Database Connection 

## Embedding with Chroma DB 

In [6]:
import os
import numpy as np 
from pathlib import Path
from langchain.vectorstores import Chroma  
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders.pdf import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import  OpenAI

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings
# from langchain.document_loaders import TextLoader, DirectoryLoader

C:\Users\Admin\.conda\envs\llm\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [7]:
loader = PyPDFLoader("./tmp/tmp.pdf")
pages = loader.load_and_split()
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
texts = text_spliter.split_documents(pages)
texts[8]

Document(page_content='be able to reproduce, animals must solve a continuous stream of problems during theirlives, e.g., ﬁnding food, avoiding predators, mating, and parenting. This suggests that\nhuman intelligence primarily evolved for solving everyday problems related to survival\nin the different habitats of Homo sapiens .\nArtiﬁcial Intelligence started as an attempt to reproduce parts of human intelligence\nin machines and, just like the notion of human intelligence, it is associated with a\ncertain vagueness regarding its deﬁnition, targeted problems, performance measures,\nand relations to neighboring research ﬁelds.\nRecently, AI research has been quite successful at producing systems that are gen-\neral in the sense that they can translate between many languages, play many games,\nmanipulate many objects, predict many video frames, write many texts, generate many\nimages, and diagnose many diseases.\nStill, many of the basic challenges of AGI remain unsolved. In fact, we do n

In [8]:
Path('Knowledge_space')
CACHE_DATASET = Path('cache_dir')
DATABASE_DIR = Path('Knowledge_space')
if not DATABASE_DIR.is_dir():
    DATABASE_DIR.mkdir(parents=True)
if not CACHE_DATASET.is_dir():
    CACHE_DATASET.mkdir(parents=True)

In [9]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(
    "all-MiniLM-L6-v2", 
    cache_folder = CACHE_DATASET.resolve().__str__()
)

# text_embedding_vector = embedding_model.encode([text.page_content for text in texts])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\AritraRanjanChowdhury\GEN_AI\Assignment_PDF_Question_Answer\cache_dir\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
exp_text = np.array([text.page_content for text in texts])
exp_text.shape

(1203,)

In [11]:
from langchain_community.embeddings.sentence_transformer \
import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

### Creating Vector Store 

In [12]:
vector_db=Chroma.from_documents(documents=texts,
                               embedding = embedding_function,
                               persist_directory= str(DATABASE_DIR.resolve()))

In [13]:
import chromadb 
db_client = chromadb.Client()

In [14]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-large')
sentence = "3D ActionSLAM: wearable person tracking in multi-floor environments"
instruction = "Represent the Science title:"
embeddings = model.encode([[instruction,sentence]])
print(embeddings)


TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [ ]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-xl')
sentence = "3D ActionSLAM: wearable person tracking in multi-floor environments"
instruction = "Represent the Science title:"
embeddings = model.encode([[instruction,sentence]])
print(embeddings)


## Chroma DB Client and Lang Chain Chroma 

In [15]:
import chromadb

In [16]:
DATABASE_PATH = Path( 'VectorDB')
user_1_db  = DATABASE_PATH/'user1'

In [17]:
client = chromadb.PersistentClient(path=str(DATABASE_PATH.resolve()))

In [ ]:
import chromadb
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

In [18]:
# collection = client.create_collection(name="Collection2")
collection = client.get_collection(name="Collection2")

In [19]:
print(dir(client))


['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_add', '_admin_client', '_count', '_create_system_if_not_exists', '_delete', '_get', '_get_identifier_from_settings', '_identifer_to_system', '_identifier', '_modify', '_peek', '_populate_data_from_system', '_query', '_server', '_system', '_update', '_upsert', '_validate_tenant_database', 'clear_system_cache', 'count_collections', 'create_collection', 'database', 'delete_collection', 'from_system', 'get_collection', 'get_or_create_collection', 'get_settings', 'get_version', 'heartbeat', 'list_collections', 'max_batch_size', 'reset', 'set_database', 'set_tenant', '

In [27]:
client.list_collections()[0].name

'Collection2'

In [21]:
langchain_chroma = Chroma(
    client=client,
    collection_name="Collection_2",
    embedding_function=embedding_function,
)

In [22]:
lang_chroma = langchain_chroma.from_documents(texts, embedding_function)

In [23]:
langchain_chroma.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [24]:
lang_chroma.as_retriever()

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018CBF36AD10>)

In [26]:
lang_chroma.get()

{'ids': ['25f4f96c-c584-11ee-bc99-e0d55edbb401',
  '25f4f96d-c584-11ee-959a-e0d55edbb401',
  '25f4f96e-c584-11ee-873d-e0d55edbb401',
  '25f4f96f-c584-11ee-b2f6-e0d55edbb401',
  '25f4f970-c584-11ee-813e-e0d55edbb401',
  '25f4f971-c584-11ee-a68a-e0d55edbb401',
  '25f4f972-c584-11ee-84af-e0d55edbb401',
  '25f4f973-c584-11ee-be44-e0d55edbb401',
  '25f4f974-c584-11ee-af13-e0d55edbb401',
  '25f4f975-c584-11ee-ae8d-e0d55edbb401',
  '25f4f976-c584-11ee-920c-e0d55edbb401',
  '25f4f977-c584-11ee-836b-e0d55edbb401',
  '25f4f978-c584-11ee-9891-e0d55edbb401',
  '25f4f979-c584-11ee-8a3b-e0d55edbb401',
  '25f4f97a-c584-11ee-9dcd-e0d55edbb401',
  '25f4f97b-c584-11ee-900a-e0d55edbb401',
  '25f4f97c-c584-11ee-a5ba-e0d55edbb401',
  '25f4f97d-c584-11ee-91d6-e0d55edbb401',
  '25f4f97e-c584-11ee-b973-e0d55edbb401',
  '25f4f97f-c584-11ee-9239-e0d55edbb401',
  '25f4f980-c584-11ee-a483-e0d55edbb401',
  '25f4f981-c584-11ee-8357-e0d55edbb401',
  '25f4f982-c584-11ee-930c-e0d55edbb401',
  '25f4f983-c584-11ee-9559-